In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

# 필요한 데이터

In [2]:
#LAB, PRE, PRO 확인할 data
data = pd.read_csv('/project/guri/data/merge_data.csv')

- lab - D_ITEMS.csv에 LABEL 존재
- pro - D_ITEMS.csv에 LABEL 존재
- pre - NDC 코드만 존재 (National Drug Code) [위키피디아](https://en.wikipedia.org/wiki/National_drug_code)

In [3]:
mimic_path = '/data/MIMIC_III/'
lab = pd.read_csv(mimic_path + 'D_LABITEMS.csv')
pro = pd.read_csv(mimic_path + 'D_ITEMS.csv')
pre = pd.read_csv(mimic_path + 'PRESCRIPTIONS.csv') #코드만 있기 때문에 필요 없음
폐렴환자 = pd.read_csv('../data/폐렴환자.csv')

In [4]:
labevent = pd.read_csv(mimic_path + 'LABEVENTS.csv')

In [5]:
lab

,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
0,546,51346,Blasts,Cerebrospinal Fluid (CSF),Hematology,26447-3
1,547,51347,Eosinophils,Cerebrospinal Fluid (CSF),Hematology,26451-5
2,548,51348,"Hematocrit, CSF",Cerebrospinal Fluid (CSF),Hematology,30398-2
3,549,51349,Hypersegmented Neutrophils,Cerebrospinal Fluid (CSF),Hematology,26506-6
4,550,51350,Immunophenotyping,Cerebrospinal Fluid (CSF),Hematology,NaN
...,...,...,...,...,...,...
748,749,51551,VOIDED SPECIMEN,OTHER BODY FLUID,HEMATOLOGY,NaN
749,750,51552,VOIDED SPECIMEN,STOOL,CHEMISTRY,NaN
750,751,51553,VOIDED SPECIMEN,URINE,CHEMISTRY,NaN
751,752,51554,VOIDED SPECIMEN,JOINT FLUID,HEMATOLOGY,NaN


In [25]:
labevent[(labevent['ITEMID']==50889) & (labevent['FLAG']=='abnormal') & (labevent['VALUEUOM']=='mg/dL')]

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
12818,13403,33,176176.0,50889,2116-12-23 21:33:00,3.83,3.83,mg/dL,abnormal
29641,21105,45,NaN,50889,2129-12-31 15:05:00,3.08,3.08,mg/dL,abnormal
29668,21132,45,NaN,50889,2130-01-20 12:23:00,3.31,3.31,mg/dL,abnormal
29676,21140,45,NaN,50889,2130-01-30 15:30:00,1.51,1.51,mg/dL,abnormal
29725,21189,45,NaN,50889,2130-02-17 11:15:00,0.55,0.55,mg/dL,abnormal
...,...,...,...,...,...,...,...,...,...
16307939,16237353,26224,NaN,50889,2115-06-05 00:30:00,2.65,2.65,mg/dL,abnormal
16404000,16333312,26380,158552.0,50889,2163-08-15 16:55:00,8.07,8.07,mg/dL,abnormal
16411257,16339584,26395,NaN,50889,2186-08-21 13:50:00,9.27,9.27,mg/dL,abnormal
16433851,15925635,25747,NaN,50889,2163-12-21 15:00:00,1.32,1.32,mg/dL,abnormal


In [7]:
max(labevent[(labevent['ITEMID']==50889) & (labevent['FLAG']!='abnormal') & (labevent['VALUEUOM']=='mg/dL')]['VALUENUM'])

0.5

In [46]:
labevent[(labevent['ITEMID']==50889) & (labevent['FLAG']=='abnormal') & (labevent['SUBJECT_ID'].isin(폐렴환자['SUBJECT_ID'].unique()))]['SUBJECT_ID'].value_counts()[:20]

15301    12
64334    11
11085    11
81245    10
15209     9
19038     9
41446     8
78076     8
93113     8
29027     7
25225     7
3386      7
19728     7
92233     7
30798     7
64280     6
29299     6
21350     6
18094     6
14676     6
Name: SUBJECT_ID, dtype: int64

In [62]:
tmp = labevent[(labevent['ITEMID']==50889) & (labevent['FLAG']=='abnormal') & (labevent['SUBJECT_ID'].isin(폐렴환자['SUBJECT_ID'].unique()))]['SUBJECT_ID'].unique()


In [59]:
labevent[(labevent['ITEMID']==50889) & (labevent['SUBJECT_ID']==15209)].sort_values('CHARTTIME')

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
9472415,9425589,15209,NaN,50889,2128-05-12 06:15:00,16.52,16.52,mg/dL,abnormal
9436222,9425778,15209,NaN,50889,2128-05-20 09:50:00,11.13,11.13,mg/dL,abnormal
9458158,9425936,15209,NaN,50889,2128-05-30 04:05:00,GREATER THAN 30,NaN,mg/dL,NaN
9438804,9425961,15209,141701.0,50889,2128-06-08 20:02:00,26.88,26.88,mg/dL,abnormal
9460695,9426058,15209,141701.0,50889,2128-06-11 03:33:00,10.35,10.35,mg/dL,abnormal
9441410,9426162,15209,NaN,50889,2128-08-19 17:21:00,29.90,29.90,mg/dL,abnormal
9463579,9426415,15209,NaN,50889,2128-08-28 05:00:00,4.96,4.96,mg/dL,abnormal
9446618,9426550,15209,157023.0,50889,2128-09-11 21:41:00,14.27,14.27,mg/dL,abnormal
9450603,9426936,15209,157023.0,50889,2128-09-22 15:44:00,7.77,7.77,mg/dL,abnormal
9455615,9427088,15209,157023.0,50889,2128-09-29 05:30:00,5.22,5.22,mg/dL,abnormal


In [64]:
폐렴환자[폐렴환자['SUBJECT_ID'].isin(tmp)]

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
4,628,665,M,2052-05-20 00:00:00,2120-02-04 00:00:00,2120-02-04 00:00:00,NaN,1
9,650,689,F,2128-12-27 00:00:00,NaN,NaN,NaN,0
23,738,784,F,2131-08-01 00:00:00,NaN,NaN,NaN,0
30,772,818,F,2120-10-11 00:00:00,NaN,NaN,NaN,0
43,32,36,M,2061-08-17 00:00:00,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...
7744,45806,97212,F,2054-03-20 00:00:00,NaN,NaN,NaN,0
7748,45820,97273,M,2123-06-05 00:00:00,NaN,NaN,NaN,0
7765,40974,78707,F,2081-12-28 00:00:00,NaN,NaN,NaN,0
7785,38815,70234,M,2088-01-17 00:00:00,NaN,NaN,NaN,0


# FI Top N

In [4]:
fi_all = pd.read_csv('./reverse_FI.csv')
fi_d1d5 = pd.read_csv('./reverse_FI_d1d5.csv')
fi_d6d10 = pd.read_csv('./reverse_FI_d6d10.csv')

In [5]:
n = 10

fi_all_top = fi_all.feature[:10].to_list()
fi_d1d5_top = fi_d1d5.feature[:10].to_list()
fi_d6d10_top = fi_d6d10.feature[:10].to_list()

print(fi_all_top)
print(fi_d1d5_top)
print(fi_d6d10_top)

['409606211', '51479005520', '67434504', '74606211', '59011010325', '54858516', '50927', '409115170', '83000730', '2411260']
['409606211', '51479005520', '74606211', '67434504', '59011010325', '54858516', '50927', '409115170', '83000730', '2411260']
['51479005520', '409606211', '10019021302', '51148', '83000730', '59011010325', '49669521303', '121048910', '173024256', '54001920']


# 살펴보기
1. data에서 LAB/PRE/PRO 확인
2. 맞는 df에서 라벨 확인 or 검색

In [4]:
def in_pro(item):
    return pro[pro['ITEMID'] == item]
def in_lab(item):
    return lab[lab['ITEMID'] == item]

In [5]:
in_lab(50927)

,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
254,128,50927,Gamma Glutamyltransferase,Blood,Chemistry,2324-2


In [6]:
data[data['ITEM'].isin(map(int, fi_all_top))]

,SUBJECT_ID,DATE,ITEM,Type
1711,21,2135-02-06,74606211,PRE
1719,21,2135-02-07,74606211,PRE
1727,21,2135-02-08,74606211,PRE
2141,31,2108-08-30,74606211,PRE
2142,31,2108-08-30,74606211,PRE
...,...,...,...,...
9198994,99464,2146-04-10,409606211,PRE
9199001,99464,2146-04-11,409606211,PRE
9211008,99817,2166-11-04,50927,LAB
9211814,99817,2166-11-20,409606211,PRE


In [15]:
pre[pre['NDC'].isin(map(int, fi_all_top))][['DRUG', 'DRUG_NAME_POE', 'DRUG_NAME_GENERIC', 'NDC']]

,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,NDC
946,Olanzapine,Olanzapine,Olanzapine,2411260.0
954,Olanzapine,Olanzapine,Olanzapine,2411260.0
1438,Morphine Sulfate,NaN,NaN,74606211.0
1679,Morphine Sulfate,NaN,NaN,74606211.0
1682,Morphine Sulfate,NaN,NaN,74606211.0
...,...,...,...,...
4154523,Morphine Sulfate,NaN,NaN,409606211.0
4154538,Morphine Sulfate,NaN,NaN,409606211.0
4155253,Morphine Sulfate,NaN,NaN,409606211.0
4155259,Morphine Sulfate,NaN,NaN,409606211.0


## FIND ITEMID LABEL

### LAB

In [24]:
lab[lab['ITEMID']==50878  ]

,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
205,79,50878,Asparate Aminotransferase (AST),Blood,Chemistry,1920-8


In [ ]:
6598-7
Albumin (1751-7)
PT(5902-2)
Asparate Aminotransferase (AST) (1920-8)

### PRO

In [19]:
pro['CATEGORY'].value_counts()

Free Form Intake            2420
Access Lines - Invasive      312
ORGANISM                     312
Skin - Impairment            271
Labs                         148
                            ... 
Meds                           1
VBG'S                          1
Research Enrollment Note       1
NG Feeding                     1
Quick Admit                    1
Name: CATEGORY, Length: 94, dtype: int64

In [23]:
pro[pro['ITEMID']==224275]

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
9345,13081,224275,20 Gauge,20 Gauge,metavision,procedureevents_mv,Access Lines - Peripheral,None,Process,NaN


In [62]:
tmp = pro.fillna('nan')
tmp[tmp['LABEL'].str.contains('Gauge')]

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
9344,13080,224274,22 Gauge,22 Gauge,metavision,procedureevents_mv,Access Lines - Peripheral,None,Process,nan
9345,13081,224275,20 Gauge,20 Gauge,metavision,procedureevents_mv,Access Lines - Peripheral,None,Process,nan
9346,13082,224276,16 Gauge,16 Gauge,metavision,procedureevents_mv,Access Lines - Peripheral,None,Process,nan
9347,13083,224277,18 Gauge,18 Gauge,metavision,procedureevents_mv,Access Lines - Peripheral,None,Process,nan
9424,14740,227369,22 Gauge Dressing Occlusive,22 Gauge Dressing Occlusive,metavision,chartevents,Access Lines - Peripheral,nan,Checkbox,nan
9838,13183,224119,22 Gauge Site Appear,22 Gauge Site Appear,metavision,chartevents,Access Lines - Peripheral,nan,Text,nan
9839,13184,224121,22 Gauge Reason Discontinued,22 Gauge Reason Discontinued,metavision,chartevents,Access Lines - Peripheral,nan,Text,nan
9957,14879,227365,14 Gauge Dressing Occlusive,14 Gauge Dressing Occlusive,metavision,chartevents,Access Lines - Peripheral,nan,Checkbox,nan
9958,14880,227366,16 Gauge Dressing Occlusive,16 Gauge Dressing Occlusive,metavision,chartevents,Access Lines - Peripheral,nan,Checkbox,nan
9959,14881,227367,18 Gauge Dressing Occlusive,18 Gauge Dressing Occlusive,metavision,chartevents,Access Lines - Peripheral,nan,Checkbox,nan


In [57]:
proevent = pd.read_csv(mimic_path + 'PROCEDUREEVENTS_MV.csv')

In [58]:
proevent

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,VALUE,VALUEUOM,LOCATION,...,ORDERCATEGORYNAME,SECONDARYORDERCATEGORYNAME,ORDERCATEGORYDESCRIPTION,ISOPENBAG,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE
0,379,29070,115071,232563.0,2145-03-12 23:04:00,2145-03-12 23:05:00,225401,1.0,None,NaN,...,Procedures,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN
1,380,29070,115071,232563.0,2145-03-12 23:04:00,2145-03-12 23:05:00,225454,1.0,None,NaN,...,Procedures,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN
2,381,29070,115071,232563.0,2145-03-12 23:05:00,2145-03-18 20:01:00,225792,8456.0,hour,NaN,...,Ventilation,NaN,Task,1,0,0,FinishedRunning,NaN,NaN,NaN
3,382,29070,115071,232563.0,2145-03-12 23:36:00,2145-03-12 23:37:00,225402,1.0,None,NaN,...,Procedures,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN
4,383,29070,115071,232563.0,2145-03-13 01:27:00,2145-03-16 16:00:00,224560,5193.0,min,Right IJ,...,Invasive Lines,NaN,Task,1,0,0,FinishedRunning,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258061,257337,41958,182711,246007.0,2155-08-09 01:51:00,2155-08-10 11:20:00,225204,2009.0,min,Right Antecube,...,Invasive Lines,NaN,Task,1,0,0,FinishedRunning,NaN,NaN,NaN
258062,257338,41958,182711,246007.0,2155-08-09 01:57:00,2155-08-09 01:58:00,225402,1.0,None,NaN,...,Procedures,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN
258063,257339,41958,182711,246007.0,2155-08-09 01:57:00,2155-08-09 01:58:00,225459,1.0,None,NaN,...,Imaging,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN
258064,257340,41958,182711,246007.0,2155-08-09 01:57:00,2155-08-09 01:58:00,225966,1.0,None,NaN,...,Procedures,NaN,Electrolytes,0,0,1,Rewritten,NaN,RN,2155-08-09 02:20:00


In [22]:
proevent[proevent['ITEMID']==224275]

NameError: name 'proevent' is not defined

### PRE

In [14]:
pre[pre['NDC']==55390000401 ].iloc[:,6:]

,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
14174,MAIN,Glucagon,Glucagon,Glucagon,GLUC1I,041660,5.539000e+10,1mg Vial,1,mg,1,VIAL,IM
16581,MAIN,Glucagon,Glucagon,Glucagon,GLUC1I,041660,5.539000e+10,1mg Vial,1,mg,1,VIAL,IM
16999,MAIN,Glucagon,Glucagon,Glucagon,GLUC1I,041660,5.539000e+10,1mg Vial,1,mg,1,VIAL,IM
17513,MAIN,Glucagon,Glucagon,Glucagon,GLUC1I,041660,5.539000e+10,1mg Vial,1,mg,1,VIAL,IM
18451,MAIN,Glucagon,Glucagon,Glucagon,GLUC1I,041660,5.539000e+10,1mg Vial,1,mg,1,VIAL,IM
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4155971,MAIN,Glucagon,Glucagon,Glucagon,GLUC1I,041660,5.539000e+10,1mg Vial,1,mg,1,VIAL,IM
4156239,MAIN,Glucagon,Glucagon,Glucagon,GLUC1I,041660,5.539000e+10,1mg Vial,1,mg,1,VIAL,IM
4156247,MAIN,Glucagon,Glucagon,Glucagon,GLUC1I,041660,5.539000e+10,1mg Vial,1,mg,1,VIAL,IM
4156359,MAIN,Glucagon,Glucagon,Glucagon,GLUC1I,041660,5.539000e+10,1mg Vial,1,mg,1,VIAL,IM


In [21]:
pre[pre['NDC']==224275 ]['DRUG'].unique()

array([], dtype=object)

In [16]:
pre[pre['NDC']==591040401]['PROD_STRENGTH'].unique()

array(['40mg Tab', '40 mg Tab'], dtype=object)

In [22]:
pre[(pre['NDC']==781210301) & (pre['DRUG']=='Tacrolimus Suspension')]

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
2455168,3291945,40828,155588,243271.0,2117-02-17 00:00:00,2117-02-18 00:00:00,MAIN,Tacrolimus Suspension,Tacrolimus Suspension,Tacrolimus,TACR1,021796,781210301.0,1 mg Capsule,4,mg,4,CAP,PO
2608032,3327812,48123,111089,NaN,NaN,NaN,MAIN,Tacrolimus Suspension,Tacrolimus Suspension,Tacrolimus (Self Med),TACR1SM,021796,781210301.0,1 mg Capsule,3,mg,3,CAP,PO
2609608,3327811,48123,111089,207812.0,2173-11-04 00:00:00,2173-11-05 00:00:00,MAIN,Tacrolimus Suspension,Tacrolimus Suspension,Tacrolimus (Self Med),TACR1SM,021796,781210301.0,1 mg Capsule,3,mg,3,CAP,PO
3109285,305399,62833,181806,272472.0,2122-01-16 00:00:00,2122-01-17 00:00:00,MAIN,Tacrolimus Suspension,Tacrolimus Suspension,Tacrolimus,TACR1,21796.0,781210301.0,1 mg Capsule,1,mg,1,CAP,PO
3518091,480357,79051,111389,299803.0,2118-02-07 00:00:00,2118-02-10 00:00:00,MAIN,Tacrolimus Suspension,Tacrolimus Suspension,Tacrolimus,TACR1,021796,781210301.0,1 mg Capsule,2,mg,2,CAP,PO
3518092,480358,79051,111389,299803.0,2118-02-07 00:00:00,2118-02-10 00:00:00,MAIN,Tacrolimus Suspension,Tacrolimus Suspension,Tacrolimus,TACR1,021796,781210301.0,1 mg Capsule,3,mg,3,CAP,PO
3936531,2971543,90403,103315,249503.0,2180-10-01 00:00:00,2180-10-01 00:00:00,MAIN,Tacrolimus Suspension,Tacrolimus Suspension,Tacrolimus,TACR1,021796,781210301.0,1 mg Capsule,2,mg,2,CAP,PO
3974672,4081547,93321,115396,NaN,2128-08-30 00:00:00,2128-08-30 00:00:00,MAIN,Tacrolimus Suspension,Tacrolimus Suspension,Tacrolimus,TACR1,021796,781210301.0,1 mg Capsule,1,mg,1,CAP,PO
3974677,4081549,93321,115396,NaN,2128-08-30 00:00:00,2128-09-02 00:00:00,MAIN,Tacrolimus Suspension,Tacrolimus Suspension,Tacrolimus,TACR1,021796,781210301.0,1 mg Capsule,1,mg,1,CAP,PO
4107484,969714,98573,175201,NaN,NaN,NaN,MAIN,Tacrolimus Suspension,Tacrolimus Suspension,Tacrolimus (Self Med),TACR1SM,021796,781210301.0,1 mg Capsule,3,mg,3,CAP,PO


In [21]:
pre[pre['NDC']==781210301]['DRUG'].unique()

array(['Tacrolimus', 'Tacrolimus Suspension', 'Tacrolimus (Self Med)',
       'Tacroli', 'Tacro', 'Tacrolimus '], dtype=object)

In [44]:
pre[pre['NDC']==409606211]['FORMULARY_DRUG_CD'].unique()

array(['MORP100PM'], dtype=object)

In [46]:
pre[pre['NDC']==54858516]['FORMULARY_DRUG_CD'].unique()

array(['MOR10L'], dtype=object)

In [48]:
pre['FORMULARY_DRUG_CD'].nunique()

3267

In [52]:
data[data['Type']=='PRE'].ITEM.nunique()

3667

In [53]:
data[data['Type']=='LAB'].ITEM.nunique()

288

In [54]:
data[data['Type']=='PRO'].ITEM.nunique()

114

In [39]:
pre[pre['NDC']==0]['DRUG'].unique()

array(['D5W', 'Insulin', 'Iso-Osmotic Dextrose', 'SW', 'Send 500mg Vial',
       'Syringe (Neonatal) *D5W*', 'Soln.', 'Syringe (IV Room)', 'Vial',
       'Dextrose 5%', 'NS', 'Normocarb', 'Maalox/Benadryl/Lidocaine',
       'Sodium Chloride 0.9%  Flush', 'Syringe',
       'NEO*IV*Parenteral Nutrition', 'Syringe (Neonatal)',
       'Isotonic Sodium Chloride', 'Maalox/Diphenhydramine/Lidocaine',
       'Iso-Osmotic Sodium Chloride', '0.9% Sodium Chloride', 'Soln',
       '5% Dextrose', 'Solution', 'Bag', 'AMP', 'Syringe (PCA)',
       'Prismasate (B32 K2)', 'Syringe (Neonatal) *SW*', '1/2 NS',
       'D7.5W', 'Syringe (Neonatal) *NS*', 'Sodium CITRATE 4%',
       'Papain 2.5 % Solution', 'D5W (Glass Bottle)', 'NS (Syringe)',
       'Sterile Water', 'Prismasate (B22 K4)',
       'Trisodium Citrate 0.4% Replacement Soln',
       'Normocarb Dialysis Soln', 'Syringe (Chemo)', 'Sodium Citrate 4%',
       'NS Epidural Bag ', 'Sterile Diluent for Flolan', 'Nesiritide',
       'TRISODIUM CITRATE

In [33]:
pre[pre['DRUG']=='Insulin']['NDC'].unique()

array([0.00000000e+00, 8.82220330e+07, 2.83150100e+06, 2.82150100e+06,
       2.75100100e+06, 2.75110100e+06, 2.87150100e+06, 5.48684381e+10,
       1.69183311e+08, 2.84150100e+06,            nan, 2.86150100e+06,
       2.85010100e+06, 1.69183411e+08, 1.69750111e+08, 8.88120044e+09,
       1.69368512e+08])

In [9]:
pre[pre['DRUG']=='Morphine Sulfate'].iloc[:,8:15]

,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX
45,Morphine Sulfate,Morphine Sulfate,MORP2I,004070,74176201.0,2mg Syringe,0.5-4.0
173,Morphine Sulfate,Morphine PCA,MORP50PCA,004067,338268975.0,50mg/50mL Syringe,1
261,Morphine Sulfate,Morphine Sulfate (2-4mg),MORP2-4,004070,74176230.0,2-4mg Range,2-4
295,Morphine Sulfate,Morphine Sulfate (1-5mg),MORP1-5,004073,74125901.0,1-5mg Range,1-5
377,Morphine Sulfate,Morphine Sulfate,MORP2I,004070,74176230.0,2mg Syringe,0.5-4
...,...,...,...,...,...,...,...
4156153,Morphine Sulfate,Morphine Sulfate (Syringe),MORP2I,004070,409176230.0,2mg Syringe,0.1-0.5
4156157,Morphine Sulfate,Morphine Sulfate (Syringe),MORP2I,004070,409176230.0,2mg Syringe,0.25
4156161,Morphine Sulfate,Morphine Sulfate (Syringe),MORP2I,004070,409176230.0,2mg Syringe,0.5
4156163,Morphine Sulfate,Morphine Sulfate (Syringe),MORP2I,004070,409176230.0,2mg Syringe,0.25


In [10]:
# DRUG, DRUG_NAME_POE, DRUG_NAME_GENERIC => 다양한 표현방식
pre[pre['NDC']==0][['DRUG', 'DRUG_NAME_POE', 'DRUG_NAME_GENERIC', 'NDC']]

,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,NDC
3,D5W,NaN,NaN,0.0
7,D5W,NaN,NaN,0.0
10,D5W,NaN,NaN,0.0
16,D5W,NaN,NaN,0.0
23,D5W,NaN,NaN,0.0
...,...,...,...,...
4156421,SW,NaN,NaN,0.0
4156422,SW,NaN,NaN,0.0
4156423,Insulin,Insulin,Insulin - Sliding Scale,0.0
4156426,Sodium Chloride 0.9% Flush,Sodium Chloride 0.9% Flush,Sodium Chloride 0.9% Flush,0.0


In [11]:
pre['DRUG'].nunique()

4525

In [12]:
pre['NDC'].nunique()

4204

# 각 피쳐 전체 1의 합 확인

In [65]:
COLS = list(pd.read_csv('/project/LSH/total_data_7727.csv')['ITEMID'].sort_values().unique())
X = np.load('/project/LSH/x_(7727,10,4068).npy')

In [70]:
result = []
#4068 - ITEM
for f in range(X.shape[-1]):
    d_sum = X[:,:,f].sum()
    result.append({'cols':COLS[f],'sum':d_sum})

In [75]:
feature_sum = pd.DataFrame(result).sort_values('sum', ascending=False)

In [85]:
feature_sum[:10]

,cols,sum
0,0,53224.0
207,51279,50346.0
175,51222,49860.0
174,51221,49837.0
71,50931,39093.0
121,51006,32369.0
217,51301,26995.0
206,51277,25837.0
41,50893,23831.0
203,51274,22678.0


In [79]:
feature_sum[feature_sum['cols']==50889]

,cols,sum
37,50889,345.0
